In [1]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0) 
display(controller)

Controller()

In [29]:
import RPi.GPIO as gpio
import time

enA = 17
in1 = 27
in2 = 22

enB = 25
in3 = 23
in4 = 24

pwmA = None
pwmB = None
dutyCycleA = 50 
dutyCycleB = 50 

In [30]:
def init():    
    global pwmA, pwmB
    gpio.setmode(gpio.BCM)
    gpio.setup(enA, gpio.OUT)
    gpio.setup(in1, gpio.OUT)
    gpio.setup(in2, gpio.OUT)
    
    gpio.setup(in3, gpio.OUT)
    gpio.setup(in4, gpio.OUT)
    gpio.setup(enB, gpio.OUT)

    
    pwmA = gpio.PWM(enA, 100)  
    pwmA.start(0)

    pwmB = gpio.PWM(enB, 100)  
    pwmB.start(0)

In [31]:
def stop_robot_motion():
    pwmA.stop()
    pwmB.stop()
    gpio.cleanup() 

def stop_motion():
    pwmA.ChangeDutyCycle(0)
    pwmB.ChangeDutyCycle(0)

def forward(dutyCycleA, dutyCycleB):
    pwmA.ChangeDutyCycle(dutyCycleA)
    pwmB.ChangeDutyCycle(dutyCycleB)
    
    gpio.output(in1, False)
    gpio.output(in2, True)
    
    gpio.output(in3, False)
    gpio.output(in4, True)
    
def reverse(dutyCycleA, dutyCycleB):
    pwmA.ChangeDutyCycle(dutyCycleA)
    pwmB.ChangeDutyCycle(dutyCycleB)
    
    gpio.output(in1, True)
    gpio.output(in2, False)
    
    gpio.output(in3, True)
    gpio.output(in4, False)
    
def left_turn(dutyCycleA, dutyCycleB):
    pwmA.ChangeDutyCycle(dutyCycleA)
    pwmB.ChangeDutyCycle(dutyCycleB) 

    # Motorul drept se roteste spre fata
    gpio.output(in1, True)
    gpio.output(in2, False)

    # Motorul stang se roteste spre spate
    gpio.output(in3, False)
    gpio.output(in4, True)
    
def right_turn(dutyCycleA, dutyCycleB):
    pwmA.ChangeDutyCycle(dutyCycleA)  
    pwmB.ChangeDutyCycle(dutyCycleB)

    # Motorul stang se roteste spre spate
    gpio.output(in1, False) 
    gpio.output(in2, True)

    # Motorul drept se roteste spre fata
    gpio.output(in3, True)
    gpio.output(in4, False)

In [32]:
def robot_motion():
    while 1:
        
        if controller.axes[1].value <= 0.1:
                if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                    and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                    stop_motion()
                else:
                    stanga = (-controller.axes[1].value + controller.axes[0].value) / 2
                    dreapta = (-controller.axes[1].value - controller.axes[0].value) / 2

                    if stanga >= 0 and dreapta >= 0:
                        forward(stanga * 100, dreapta * 100)
                    elif stanga <= 0 and dreapta <= 0:
                        reverse(abs(stanga) * 100, abs(dreapta) * 100)
                    elif stanga <= 0 and dreapta >= 0:
                        right_turn(abs(stanga) * 100, dreapta * 100)
                    else:
                        left_turn(stanga * 100, abs(dreapta) * 100)
                        
                    # robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
                time.sleep(0.01)
        else:
                stanga = (-controller.axes[1].value - controller.axes[0].value) / 2
                dreapta = (-controller.axes[1].value + controller.axes[0].value) / 2

                if stanga >= 0 and dreapta >= 0:
                    forward(stanga * 100, dreapta * 100)
                elif stanga <= 0 and dreapta <= 0:
                    reverse(abs(stanga) * 100, abs(dreapta) * 100)
                elif stanga <= 0 and dreapta >= 0:
                    right_turn(abs(stanga) * 100, dreapta * 100)
                else:
                    left_turn(stanga * 100, abs(dreapta) * 100)
                
                # robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
                time.sleep(0.01)

In [35]:
import threading

init()
thread1 = threading.Thread(target=robot_motion)
thread1.setDaemon(True)
thread1.start()


/tmp/ipykernel_20619/466402992.py:4: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  gpio.setup(enA, gpio.OUT)
/tmp/ipykernel_20619/466402992.py:5: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  gpio.setup(in1, gpio.OUT)
/tmp/ipykernel_20619/466402992.py:6: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  gpio.setup(in2, gpio.OUT)
/tmp/ipykernel_20619/466402992.py:8: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  gpio.setup(in3, gpio.OUT)
/tmp/ipykernel_20619/466402992.py:9: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  gpio.setup(in4, gpio.OUT)
/tmp/ipykernel_20619/466402992.py:10: RuntimeWarning: This channel is already in use, cont

In [36]:
stop_robot_motion()